In [1]:
import numpy as np;
from MkGRASP import write_coord,write_surf,write_rim,write_scatter_multer;
from MkGRASP import write_frequency,write_beam,write_multi_po;
from MkGRASP import write_planar_grid,write_spherical_grid;

In [2]:
''' 
basic information of two mirrors
'''
#1. size of the two mirrors
M2_size=[400,437]
M1_size=[400,437]
List_m2=np.array([[0,1.5]]);
List_m1=np.array([[0,1.5]]);
#2. source distance
distance=8;
#3. source range
u_range=0.1
v_range=0.1
Nu=101;
Nv=101;
# setup of telescope
Theta_0=0.927295218001612
Ls=780.0470675116178
Lm=450
F=1300
a=585.1235655326061
L_fimag=a*2+Ls;
Feq=960
# off-axis angle
u0=2.15/180*np.pi;
v0=2.15/180*np.pi;
u0=0.024
v0=0.024
defocus=[0,0,0];

IFsize=[-500,500,101]
plane1=[-400,400,101];
plane2=[-400,400,101];

defocus_fimag=[0,0,0];
defocus_fimag[2]=1/(1/F-1/(Ls+defocus[2]))+L_fimag;
defocus_fimag[1]=(F+L_fimag-defocus_fimag[2])/F*defocus[1];
defocus_fimag[0]=(F+L_fimag-defocus_fimag[2])/F*defocus[0];
D_fimag=[0,0,0]
D_fimag[0]=defocus_fimag[0];
D_fimag[1]=defocus_fimag[1]*np.cos(Theta_0)-np.sin(Theta_0)*(L_fimag-defocus_fimag[2]+Lm);
D_fimag[2]=-defocus_fimag[1]*np.sin(Theta_0)-np.cos(Theta_0)*(L_fimag-defocus_fimag[2]);
defocus_angle=[-np.arctan(4/3)*180/np.pi,0,0];

In [3]:
'''1. make coords'''
class coord():
    def __init__(self,name='',origin=[0,0,0],angle=[0,0,0],base='mechanical_axis.cs',Type='coor_sys'):
        self.name=name;
        self.origin=np.array(origin);
        self.angle=np.array(angle)/180*np.pi;
        self.base=base;
        self.str=write_coord(self.name,self.origin,self.angle,base=self.base,Type=Type);

secondary_mirror=coord(name='secondary_mirror.cs',origin=[0,-Lm*np.sin(Theta_0),0],angle=[np.arctan(3/4)/2*180/np.pi-90,0,0],Type='coor_sys');
primary_mirror=coord(name='primary_mirror.cs',origin=[0,0,Lm*np.cos(Theta_0)],angle=[-np.arctan(4/3)/2*180/np.pi,0,0],Type='coor_sys');
focale_plane=coord(name='focale_plane.cs',origin=[0,Ls-Lm*np.sin(Theta_0),0],angle=[-90,0,0],Type='coor_sys');
focale_plane_rev=coord(name='focale_plane_rev.cs',origin=defocus,angle=[180,0,0],base='focale_plane.cs',Type='coor_sys')
mechanical_axis=coord(name='mechanical_axis.cs');
boresight=coord(name='boresight.cs',origin=[0,0,0],angle=[0,180,0],Type='coor_sys');
print(defocus_angle);
IF_coord=coord(name='IF_coord.cs',origin=D_fimag,angle=defocus_angle,Type='coor_sys');

'''2. make panel rims'''
def rims(centres,sizes,name=''):
    Str='';
    items=();
    for n in range(int(centres.size/2)):
        Str+=write_rim(name+'_'+str(n+1).zfill(2),centres[n,...],sizes[n,...],Type='rectangular_rim');
        items=items+((name+'_'+str(n+1).zfill(2))+'.rim',)
    return Str,items;



M2_size=M2_size*int(List_m2.size/2);
M1_size=M1_size*int(List_m1.size/2);
M2_rims,M2_items=rims(List_m2,np.array(M2_size).reshape(-1,2),name='secondary');
M1_rims,M1_items=rims(List_m1,np.array(M1_size).reshape(-1,2),name='primary');

'''3. make surface'''
secondary_mirror_surf=write_surf('secondary_mirror',filename='surf/secondary_surface.sfc',Type='regular_xy_grid');
primary_mirror_surf=write_surf('primary_mirror',filename='surf/primary_surface.sfc',Type='regular_xy_grid');
#IF_plane=write_surf_plane('IF_plane',point);
'''4. make panel distortion surface'''

'''5. make scatters'''
primary_panels=write_scatter_multer('primary_panels','primary_mirror.cs','primary_mirror.surf',M1_items,distortions='')
secondary_panels=write_scatter_multer('secondary_panels','secondary_mirror.cs','secondary_mirror.surf',M2_items,distortions='');
'''6. make frequency'''
feed_296=write_frequency('feed_296',freq_list=[296]);

'''7. make Gaussian feed'''
gaussian_beam_pattern=write_beam('gaussian_beam_pattern',freq='feed_296',coor_sys='focale_plane_rev.cs',taper_angle=11.894,taper=-8,polarisation='linear_x');


'''8. make po_multi_face_scatterer'''
primary_panels_po=write_multi_po('primary_panels','feed_296','primary_panels.refl',method='po');
secondary_panels_po=write_multi_po('secondary_panels','feed_296','secondary_panels.refl',method='po');

'''9. make source filed'''
near_uv=write_spherical_grid('near_uv','boresight.cs',u_range,v_range,0,0,Nu,Nv,near_far='near',near_dist=distance,filename='');
near_uv_pospos=write_spherical_grid('near_uv_pospos','boresight.cs',u_range,v_range,u0,-v0,Nu,Nv,near_far='near',near_dist=distance,filename='');
near_uv_posneg=write_spherical_grid('near_uv_posneg','boresight.cs',u_range,v_range,u0,v0,Nu,Nv,near_far='near',near_dist=distance,filename='');
near_uv_negpos=write_spherical_grid('near_uv_negpos','boresight.cs',u_range,v_range,-u0,-v0,Nu,Nv,near_far='near',near_dist=distance,filename='');
near_uv_negneg=write_spherical_grid('near_uv_negneg','boresight.cs',u_range,v_range,-u0,v0,Nu,Nv,near_far='near',near_dist=distance,filename='');

near_IF=write_planar_grid('near_IF','IF_coord.cs',0,x_range=IFsize,y_range=IFsize,grid_type='xy',filename='')
near_plane1=write_planar_grid('near_plane1','boresight.cs',near_dist=7,x_range=plane1,y_range=plane1,grid_type='xy',filename='')
near_plane2=write_planar_grid('near_plane2','boresight.cs',near_dist=8,x_range=plane2,y_range=plane2,grid_type='xy',filename='')

[-53.13010235415598, 0, 0]


In [4]:
# write to tor file;
with open('torfile/CCAT-optics.tor','w') as f:
    '''1. write geometry'''
    # write coordinates systems;
    f.writelines(mechanical_axis.str);
    f.writelines(secondary_mirror.str);
    f.writelines(primary_mirror.str);
    f.writelines(focale_plane.str);
    f.writelines(focale_plane_rev.str);
    f.writelines(boresight.str);
    f.writelines(IF_coord.str);
    # write surf
    f.writelines(secondary_mirror_surf);
    f.writelines(primary_mirror_surf);
    # write rim
    f.writelines(M2_rims);
    f.writelines(M1_rims);
    # write scatter
    f.writelines(primary_panels)
    f.writelines(secondary_panels);
    # write frequency
    f.writelines(feed_296);
    # write gaussian feed
    f.writelines(gaussian_beam_pattern);
    
    
    '''2. write po'''
    f.writelines(primary_panels_po);
    f.writelines(secondary_panels_po);
    
    '''3. write field'''
    # write source field
    f.writelines(near_uv);
    f.writelines(near_uv_pospos);
    f.writelines(near_uv_posneg);
    f.writelines(near_uv_negpos);
    f.writelines(near_uv_negneg);
    # write plane field;
    f.writelines(near_IF);
    f.writelines(near_plane1);
    f.writelines(near_plane2);
    
    
    
    

In [5]:
[0,Ls-Lm*np.sin(Theta_0),0]

[0, 420.04706751161785, 0]

In [6]:
[0,-Lm*np.sin(Theta_0),0]

[0, -359.99999999999994, 0]

In [7]:
[0,0,Lm*np.cos(Theta_0)]

[0, 0, 270.0000000000001]